# Creating the Lollapalooza charts from Déborah Mesquita's tutorial

[Tutorial on freecodecamp](https://www.freecodecamp.org/news/how-and-why-i-used-plotly-instead-of-d3-to-visualize-my-lollapalooza-data-d48345e2ca68/)

The following text is copied from the article and placed in this notebook to make it easier for you to render the charts.

There is no video explanation for this activity as you already completed it once last week for the plotly express version of the charts.

## My Lollapalooza experience

For the 2018 edition of Lollapalooza Brazil, all purchases were made through an RFID-enabled wristband. They send the data to your email address, so I decided to take a look at it. What can we learn about me and my experience by analyzing the purchases I made at the festival?

This is how the data looks:

- purchase date
- purchase hour
- product
- quantity
- stage
- place where I did the purchase

Based on this data, let’s answer some questions.

## Where did I go during the festival?

The data only tells us the name of the location where I made the purchase, and the festival took place at Autódromo de Interlagos. 

I took the map with the stages from here and used the georeferencer tool from georeference.com to get the latitude and longitude coordinates for the stages.

We need to display a map and the markers for each purchase, so we will use Mapbox and the scattermapbox trace. 

You will need to [generate a mapbox token from the mapbox site](https://www.mapbox.com/help/define-access-token/) and then place your token in the first line of code after the import statements in the cell below. You will need to create an account to do this. If you don't feel comfortable in signing up then you can skip this chart and move to the next.

First let’s plot only the stages to see how this works:

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"

mapbox_token = "" # Add your mapbox token here

df = pd.read_csv("../data/stages.csv")

trace = go.Scattermapbox(lat = df["latitude"], lon = df["longitude"], text=df["stage"], marker=go.scattermapbox.Marker(size=10), mode="markers+text", textposition="top left")

data = [trace]

layout = go.Layout(mapbox=dict(accesstoken=mapbox_token, center=dict(lat = -23.701057,lon = -46.6970635), zoom=14.5))

figure = go.Figure(data=data, layout= layout)

figure.show()

## How did I spend my money?
To answer that, I created a bar chart with my spendings for food and beverage by each day and built a heatmap to show when I bought stuff. 

We already saw how to build a bar chart (in the previous exercise):

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"

df = pd.read_csv("../data/purchase_data.csv")

df_purchases_by_place = df.pivot_table(index="place",columns="date",values="price",aggfunc="sum").fillna(0)

data = []

for index,place in df_purchases_by_place.iterrows():
    trace = go.Bar(x = df_purchases_by_place.columns, y = place, name=index)
    data.append(trace)

layout = go.Layout(title="Purchases by place", showlegend=True, barmode="stack")

figure = go.Figure(data=data, layout=layout)

figure.show()

Now let’s build a heatmap chart:

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"

df = pd.read_csv("../data/purchase_data.csv")

df_purchases_by_type = df.pivot_table(index="place",columns="date",values="price",aggfunc="sum").fillna(0)

df["hour_int"] = pd.to_datetime(df["hour"], format="%H:%M", errors='coerce').apply(lambda x: int(x.hour))

df_heatmap = df.pivot_table(index="date",values="price",columns="hour", aggfunc="sum").fillna(0)

trace_heatmap = go.Heatmap(x = df_heatmap.columns, 
                           y = df_heatmap.index, 
                           z = [df_heatmap.iloc[0], 
                                df_heatmap.iloc[1], 
                                df_heatmap.iloc[2]])

data = [trace_heatmap]

layout = go.Layout(title="Purchases by place", showlegend=True)

figure = go.Figure(data=data, layout=layout)

figure.show()

## Which concerts did I watch?
Now let’s go to the coolest part: could I guess the concerts I attended based only on my purchases?

Ideally, when we are watching a show, we are watching the show (and not buying stuff), so the purchases should be made before or after each concert. I then made a list of each concert happening one hour before, one hour after, and according to the time the purchase was made.

To find out which one of these shows I attended, I calculated the distance from the location of the purchase to each stage. The shows I attended should be the ones with the shortest distance to the concessions.

As we want to show each data point, the best choice for a visualization is a table. Let’s build one:


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"

df_table = pd.read_csv("../data/concerts_I_attended.csv")

def color_font(x):    
    if x == "Yes":       
        return "rgb(0,0,9)"    
    else:       
        return "rgb(178,178,178)"
    
df_table["color"] = df_table["correct"].apply(lambda x: color_font(x))

trace_table = go.Table(header=dict(values=["Concert","Date","Correct?"], fill=dict(color=("rgb(82,187,47)"))),
                       cells=dict(values= [df_table.concert, df_table.date,df_table.correct], font=dict(color=([df_table.color]))))

data = [trace_table]

figure = go.Figure(data = data)

figure.show()

Three concerts were missing and four were incorrect, giving us a precision of 67% and recall of 72%.
